# **SELECCIÓN DE CARACTERÍSTICAS**

In [1]:
#IMPORTACIÓN DE LIBRERÍAS

# En este apartado se realizará la importación de todas las librerías 
# a ser ocupadas para la creación del modelo como tersor flow , pandas ,sklearn 
# y numpy

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#CONEXIÓN A DRIVE Y LECTURA DEL DATASET

# El entorno de trabajo es colab por lo que es necesario importar el drive si nuestros archivos se 
# encuentran en este.

from google.colab import drive
drive.mount('/content/drive')

# Mediante la librería de pandas mandamos a buscar el archivo a la carpeta contenida en el drive
# una vez encontrado se colocaran tags a las columnas del dataframe

#El primer dataset se usa para la implementación del modelo de random forest, para la importancia de las características.
df1 = pd.read_excel("/content/drive/MyDrive/Colab/ResultadosCompleto.xlsx")

#Al primer dataset se le harán algunas modificaciones, por lo que se define un segundo dataset, con el mismo archivo. 
#Este segundo dataset se lo usa para la implementación del modelo
df = pd.read_excel("/content/drive/MyDrive/Colab/ResultadosCompleto.xlsx")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#INFORMACIÓN DEL DATASET
# Con el comando "info()", se obtiene el tipo de datos de las columnas y el número total de registros
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   edad          2210 non-null   int64 
 1   genero        2210 non-null   object
 2   personalidad  2210 non-null   object
 3   estilo        2210 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 69.2+ KB


In [4]:
#VERIFICACIÓN DE VALORES NULOS EN EL DATASET

#Se debe comprobar que en el dataset no existan valores nulos, para que en los siguientes procesos, no se muestren resultados inconsistentes
check_for_nan = df1.isnull().values.any()
print (check_for_nan)



False


In [5]:
#TRANSFORMACIÓN DE CARACTERÍSTICAS CATEGÓRICAS A NUMÉRICAS PARA LA SELECCIÓN DE CARACTERÍSTICAS
#CON RANDOM FOREST

# Se realiza la respectiva transformación de los datos para que el modelo pueda entenderlos,
# como se puede ver en el siguiente algoritmo, todos los datos de las columnas de género y 
# personalidad se les asignará un número reemplazando los valores no numéricos

df1['genero'] = df1['genero'].map({'Masculino': 0,
                                 'Femenino': 1})

df1['personalidad'] = df1['personalidad'].map({'Confiable': 0,
                                           'Extravertido': 1,
                                           'Animado': 2,
                                           'Responsable': 3,
                                           'Serio': 4})

In [7]:
#DECLARACIÓN DE VARIABLES TEMPORALES PARA X E Y PARA LA SELECCIÓN DE CARACTERÍSTICAS

#Para las variables en X, seleccionamos todas las variables del dataset, excepto la variable que tendremos de salida: el estilo. 

X_inicial = df1.iloc[:,0:3]
y_inicial = df1.iloc[:,-1]
X_inicial

,edad,genero,personalidad
0,25,0,0
1,28,1,2
2,18,1,2
3,45,0,1
4,21,0,0
...,...,...,...
2205,26,1,2
2206,29,1,2
2207,22,0,1
2208,22,0,3


In [8]:
#El algoritmo de random forest de clasificación, contiene un comando que nos permite calcular el rango de importancia de cada variable.
#Así pues, según el resultado que se obtenga en este top, se comprueba si se está seleccionando correctamente las características. 

#En este paso, se realiza un entrenamiento parcial con las variables sin procesar. 
#Allí se consideran las variables independientes y la variable dependiente, tando de entrenamiento como de testeo. 
#Con este proceso, lo que se busca es listar a las características en orden ascendente, desde la más importante. 
#Debido a que solo se tienen 3 datos de entrada, el array consta de 3 variables. 


RF = RandomForestClassifier(max_depth=7, n_estimators=100)

df1 = pd.DataFrame()
RF.fit(X_inicial, y_inicial)
y_pred_RF= RF.predict(X_inicial)
cols = ['Machine Learning Classification Method',
        'Train Accuracy', 
        'Test Accuracy', 
        "Top 1 Feature Predictor",
        "Top 2 Feature Predictor",
        "Top 3 Feature Predictor"]

df1.loc['Machine Learning Classification Method', 
       'Details'] = 'Random Forest Classifier'
df1.loc['Train Accuracy', 'Details'] = RF.score(X_inicial, y_inicial) 
df1.loc['Test Accuracy', 'Details'] = RF.score(X_inicial,y_inicial)


feature_importances = X_inicial.columns[np.argsort(RF.feature_importances_)][-3:]
df1.loc['Top 1 Feature Predictor', 'Details'] = feature_importances[2]
df1.loc['Top 2 Feature Predictor', 'Details'] = feature_importances[1]
df1.loc['Top 3 Feature Predictor', 'Details'] = feature_importances[0]

display(df1)

,Details
Machine Learning Classification Method,Random Forest Classifier
Train Accuracy,0.425792
Test Accuracy,0.425792
Top 1 Feature Predictor,personalidad
Top 2 Feature Predictor,edad
Top 3 Feature Predictor,genero


**AHORA QUE SE TIENE UNA LISTA DE IMPORTANCIA DE LAS CARACTERÍSTICAS SE PROCEDE A LAS DEBIDAS TRANSFORMACIONES**

In [9]:
#TRANSFORMACIÓN DE EDAD CON LA TÉCNICA DE BINNING

#Se coloca el intervalo de edades que está permitido en el dataset (de 17 a 30 años, según el objetivo de análisis del proyecto)
#donde se dividirán en 5 rangos, que estarán etiquetados con los nombres de la variable "labels"
bins = np.linspace(17,30,6)
labels = ['0.1', '0.2', '0.3','0.4','0.5']
df['edad'] = pd.cut(df['edad'], bins, labels=labels)

print(bins)
print(df)

[17.  19.6 22.2 24.8 27.4 30. ]
     edad     genero  personalidad  estilo
0     0.4  Masculino     Confiable       5
1     0.5   Femenino       Animado       4
2     0.1   Femenino       Animado       6
3     NaN  Masculino  Extravertido       3
4     0.2  Masculino     Confiable       3
...   ...        ...           ...     ...
2205  0.4   Femenino       Animado      10
2206  0.5   Femenino       Animado       1
2207  0.2  Masculino  Extravertido       8
2208  0.2  Masculino   Responsable       4
2209  0.1  Masculino     Confiable       7

[2210 rows x 4 columns]


In [10]:
#VERIFICACIÓN DE LOS TIPOS DE DATOS LUEGO DE LA APLICACIÓN DE TÉCNICA DE BINNING

#Al utilizar binning para colocar los datos en rangos, se puede observar que el tipo de dato se cambió a categórico
#Esto indica que más adelante, se deberá realizar nuevamente una transformación para que se vuelva un dato de tipo numérico.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   edad          1956 non-null   category
 1   genero        2210 non-null   object  
 2   personalidad  2210 non-null   object  
 3   estilo        2210 non-null   int64   
dtypes: category(1), int64(1), object(2)
memory usage: 54.3+ KB


In [11]:
#VERIFICACIÓN DE DATOS NULOS 

#Es importante que luego de hacer una modificación de alguna variable en el dataset, se vuelva a comprobar que no existan datos nulos.
check_for_nan = df.isnull().values.any()
print (check_for_nan)

#Con el siguiente comando, se imprimen la cantidad de nulos que existen y en qué columnas se encuentran. 
#Solo en la edad existen datos nulos, ya que existían personas que no tenían la edad dentro del rango permitido; por tanto, no se les otorgó un
#intervalo con el proceso de binning y pasaron a ser datos nulos.
df.isnull().sum()

True


edad            254
genero            0
personalidad      0
estilo            0
dtype: int64

In [12]:
#COMO SI EXISTEN VALORES NULOS, SE LOS DEBE ELIMINAR

#Como los datos nulos, son de las edades que no corresponden al intervalo escogido para la muestra, se deben eliminar. 
#De esta manera, de los 2209 registros que se tenía en un inicio, se disminuyeron a 1956.
df_nulos = df[~df['edad'].isnull()].copy()
df_nulos

,edad,genero,personalidad,estilo
0,0.4,Masculino,Confiable,5
1,0.5,Femenino,Animado,4
2,0.1,Femenino,Animado,6
4,0.2,Masculino,Confiable,3
5,0.2,Masculino,Animado,10
...,...,...,...,...
2205,0.4,Femenino,Animado,10
2206,0.5,Femenino,Animado,1
2207,0.2,Masculino,Extravertido,8
2208,0.2,Masculino,Responsable,4


In [13]:
#TÉCNICA ONE_HOT_ENCONDE PARA LA TRANSFORMACIÓN DE VARIABLES CATEGÓRICAS A NUMÉRICAS
one_hot_genero = pd.get_dummies(df.genero ,prefix='genero')
one_hot_Personalidad = pd.get_dummies(df.personalidad ,prefix='personalidad')

#AL IMPLEMENTAR EL BINNING EN LA EDAD, ESTOS DATOS SE TRANSFORMAN EN CATEGÓRICOS. POR LO TANTO
#SE APLICA ONE_HOT_ENCODE TAMBIÉN A LA EDAD
one_hot_Edad = pd.get_dummies(df.edad ,prefix='edad')

#CONCATENACIÓN DE LAS VARIABLES TRANSFORMADAS
d = pd.concat([one_hot_Edad,one_hot_genero, one_hot_Personalidad,df.estilo], axis=1)
d

,edad_0.1,edad_0.2,edad_0.3,edad_0.4,edad_0.5,genero_Femenino,genero_Masculino,personalidad_Animado,personalidad_Confiable,personalidad_Extravertido,personalidad_Responsable,personalidad_Serio,estilo
0,0,0,0,1,0,0,1,0,1,0,0,0,5
1,0,0,0,0,1,1,0,1,0,0,0,0,4
2,1,0,0,0,0,1,0,1,0,0,0,0,6
3,0,0,0,0,0,0,1,0,0,1,0,0,3
4,0,1,0,0,0,0,1,0,1,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,0,0,0,1,0,1,0,1,0,0,0,0,10
2206,0,0,0,0,1,1,0,1,0,0,0,0,1
2207,0,1,0,0,0,0,1,0,0,1,0,0,8
2208,0,1,0,0,0,0,1,0,0,0,1,0,4


In [14]:
#VERIFICACIÓN DE LOS TIPOS DE DATOS QUE SE TRANSFORMARON
#Luego de realizar todas las transformaciones, obtenemos los tipos de datos que se encuentran en el dataset, para confirmar
#que ya no se necesiten más transformaciones
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   edad_0.1                   2210 non-null   uint8
 1   edad_0.2                   2210 non-null   uint8
 2   edad_0.3                   2210 non-null   uint8
 3   edad_0.4                   2210 non-null   uint8
 4   edad_0.5                   2210 non-null   uint8
 5   genero_Femenino            2210 non-null   uint8
 6   genero_Masculino           2210 non-null   uint8
 7   personalidad_Animado       2210 non-null   uint8
 8   personalidad_Confiable     2210 non-null   uint8
 9   personalidad_Extravertido  2210 non-null   uint8
 10  personalidad_Responsable   2210 non-null   uint8
 11  personalidad_Serio         2210 non-null   uint8
 12  estilo                     2210 non-null   int64
dtypes: int64(1), uint8(12)
memory usage: 43.3 KB


In [15]:
#NORMALIZACIÓN DE LA VARIABLE DE SALIDA (ESTILO)

#Como la variable de salida, no se encuentra en el mismo rango que el resto de los datos, se procede a realizar la normalización de esta variable.
#Para ello, se define una variable que contenga nuestros datos de salida
y_for_norm = df.iloc[:,-1]

#Y se realiza la normalización de Mínimo y máximo, para otorgarle valores al estilo de entre 0 y 1.
def minmax_norm(variable):
    return (variable - variable.min()) / ( variable.max() - variable.min())

y_norm = minmax_norm(y_for_norm)

print(y_norm)

0       0.444444
1       0.333333
2       0.555556
3       0.222222
4       0.222222
          ...   
2205    1.000000
2206    0.000000
2207    0.777778
2208    0.333333
2209    0.666667
Name: estilo, Length: 2210, dtype: float64


In [16]:
#CONCATENACIÓN DE TODAS LAS VARIABLES TRANSFORMADAS

#Ahora que ya se encuentran todas las variables transformadas, se procede a unirlas en una sola tabla.
d = pd.concat([one_hot_Edad,one_hot_genero, one_hot_Personalidad,y_norm], axis=1)
d

,edad_0.1,edad_0.2,edad_0.3,edad_0.4,edad_0.5,genero_Femenino,genero_Masculino,personalidad_Animado,personalidad_Confiable,personalidad_Extravertido,personalidad_Responsable,personalidad_Serio,estilo
0,0,0,0,1,0,0,1,0,1,0,0,0,0.444444
1,0,0,0,0,1,1,0,1,0,0,0,0,0.333333
2,1,0,0,0,0,1,0,1,0,0,0,0,0.555556
3,0,0,0,0,0,0,1,0,0,1,0,0,0.222222
4,0,1,0,0,0,0,1,0,1,0,0,0,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,0,0,0,1,0,1,0,1,0,0,0,0,1.000000
2206,0,0,0,0,1,1,0,1,0,0,0,0,0.000000
2207,0,1,0,0,0,0,1,0,0,1,0,0,0.777778
2208,0,1,0,0,0,0,1,0,0,0,1,0,0.333333


In [17]:
#Definidos ya todos los valores, se exporta el dataset con el que se trabajará en los siguientes procesos. 
d.to_excel("DataFrameTransformado.xlsx")

#En new_data se almacena el dataset que se acaba de exportar, para que, mediante esta variable, se llame al nuevo dataframe
new_data = pd.read_excel("DataFrameTransformado.xlsx")

In [18]:
#DEFINICIÓN DE VARIABLES DE ENTRADA Y LA VARIABLE DE SALIDA

#Ya teniendo todos los datos transformados, se puede definir "x" e "y" para trabajar con ellas, en el proceso posterior e implementarlas ya en el modelo
X = new_data.iloc[:,0:12]
y = new_data.iloc[:,-1].values

In [19]:
#DIVISIÓN DEL DATASET PARA LOS DATOS DE ENTRENAMIENTO Y LOS DE TESTEO

#Finalmente, se dividen los datos para que sean 70% para el entrenamiento y el 30% para el testeo.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size=0.3, random_state=100)